In [8]:
import os

import geopandas as gpd
import polars as pl
from dotenv import load_dotenv
from sqlalchemy import create_engine
import nest_asyncio

load_dotenv()
nest_asyncio.apply() 
URI = os.getenv("POSTGRES")
FETCH_API = os.getenv("HBASE_FETCH_API")
SEND_API = os.getenv("HBASE_SEND_API")

In [12]:
sql = """
select *
from geometry.boundary_town
where city_name = %(city_code)s and town_name = '大安區'
"""

engine = create_engine(URI)
with engine.connect() as conn:
    gdf = gpd.read_postgis(
        sql,
        con=conn.connection,
        geom_col='geometry',
        params={'city_code': '臺北市'} # 台北市
    )

/Users/syuanbo/Documents/Project/h3-aggregation/.venv/lib/python3.12/site-packages/geopandas/io/sql.py:185: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(
/Users/syuanbo/Documents/Project/h3-aggregation/.venv/lib/python3.12/site-packages/geopandas/io/sql.py:473: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  return pd.read_sql(spatial_ref_sys_sql, con)


In [16]:
from h3_toolkit import H3Toolkit
from h3_toolkit.hbase import HBaseClient


toolkit = H3Toolkit()

result = (
    toolkit
    .set_hbase_client(
        HBaseClient(
            fetch_url = FETCH_API,
            send_url = SEND_API
        )
    )
    .process_from_vector(gdf, resolution=12)
    .fetch_from_hbase(
        table_name='res12_pre_data',
        column_family='overture_roads',
        column_qualifier=['total_count']
    )
    .get_result()
)

In [17]:
import mapclassify as mc
import pandas as pd
import pydeck as pdk
from matplotlib import colormaps

INITIAL_VIEW_STATE = {
    'latitude':25.0307167767897,
    'longitude':121.5359704226339,
    'zoom':13,
    'max_zoom':16,
    'pitch':0,
    'bearing':0
}

In [18]:
def _set_color(
    data:pl.DataFrame | gpd.GeoDataFrame,
    target_col:str,
) -> pd.DataFrame | gpd.GeoDataFrame:
    """set color column for data based on target_col
    """

    # change this classifier to fit your data
    classifier = mc.NaturalBreaks(data[target_col], k=5)
    cmap = colormaps.get_cmap('Oranges')

    if isinstance(data, pl.DataFrame):
        data = data.to_pandas()
    data['interval'] = classifier.find_bin(data[target_col])
    data['color'] = data['interval'].apply(lambda x: cmap(x / data['interval'].max()))
    data['color'] = data['color'].apply(lambda c: [int(255 * i) for i in c[:3]] + [255])
    data = data.drop(columns='interval')

    return data


def show_h3(
    data:pl.DataFrame,
    target_col:str,
    h3_col='hex_id'
):
    """show h3 hexagon layer
    """

    data = data.clone()
    data = _set_color(data, target_col)
    data = data.to_dict(orient='records')

    layer_h3 = pdk.Layer(
        'H3HexagonLayer',
        data,
        get_fill_color='color',
        # get_line_color=[255, 255, 255],
        # get_elevation='p_cnt',
        get_hexagon=h3_col,
        pickable=True,
        opacity=0.4,
        stroked=False,
        filled=True,
        extruded=False,
    )

    r = pdk.Deck(
        layers=[layer_h3],
        initial_view_state=pdk.ViewState(**INITIAL_VIEW_STATE),
        tooltip={"text": f"{target_col}: {{{target_col}}}"}
    )
    return r

In [19]:
show_h3(result, 'total_count')

UFuncTypeError: ufunc 'multiply' did not contain a loop with signature matching types (dtype('<U3'), dtype('float64')) -> None